In [2]:
#add all tested fuctions

def matching(index1, index2):
    matched = False
    inverse = ""
    
    for x in index2:
        if x == "A":
            inverse =  "T" + inverse 
        elif x == "T":
            inverse =  "A" + inverse
        elif x == "G":
            inverse = "C" + inverse
        elif x == "C":
            inverse = "G" + inverse   
        else:
            inverse = "N" + inverse

#assuming N base calls are normally on the first base
#first 100 reads in both files only have Ns on first base
#this allows one base pair difference between matching indexes
#keeps down the rate of false positives 

    if containsN(index1):
        index1 = index1[1:]
        inverse = inverse[1:]
        
    elif containsN(index2):
        index1 = index1[:-1]
        inverse = inverse[:-1]
        
    elif containsN(index1) and containsN(index2):
        index1 = index1[1:-1]
        inverse = inverse[1:-1]
        
    return (inverse == index1)

def writeToBadFile(outputFile, read, reason):
    N = 1
    IH = 2
    QS = 4
    
    if (reason & N):
        read[2] = read[2] + " " + "(N)"
        
    if (reason & IH):
        read[2] = read[2] + " " + "(IH)"
        
    if (reason & QS):
        read[2] = read[2] + " " + "(QS)"
        
    with open (outputFile, "a") as out:
        out.write(read[0])
        out.write("\n")
        out.write(read[1])
        out.write("\n")
        out.write(read[2])
        out.write("\n")
        out.write(read[3])
        out.write("\n")
        
def writeToGoodFile(outputFile, read):
    with open (outputFile, "a") as out:
        out.write(read[0])
        out.write("\n")
        out.write(read[1])
        out.write("\n")
        out.write(read[2])
        out.write("\n")
        out.write(read[3])
        out.write("\n")
        
def convertPhred(letter):
    return ord(letter)-33

def meanQS(read):
    total = 0
    bp = 0
    for base in read[3]:
        total = total + convertPhred(base)
        bp = bp + 1
    return total/bp

def compareMeans(read):
    cutoffQS = 32
    return meanQS(read) >= cutoffQS

def containsN(index):
    return "N" in index

def readInLine(file):
    line = file.readline()
    return line.strip()

In [3]:
#10 Read subsets for each file
#zcat /projects/bgmp/shared/2017_sequencing/1294_S1_L008_R1_001.fastq.gz | head -n40 > R1_10r.fq
file10 = "testFiles/R1_10r.fq"
file20 = "testFiles/R2_10r.fq"
file30 = "testFiles/R3_10r.fq"
file40 = "testFiles/R4_10r.fq"


#100 Read subsets for each file
#zcat /projects/bgmp/shared/2017_sequencing/1294_S1_L008_R1_001.fastq.gz | head -n400 > R1_100r.fq
file100 = "testFiles/R1_100r.fq"
file200 = "testFiles/R2_100r.fq"
file300 = "testFiles/R3_100r.fq"
file400 = "testFiles/R4_100r.fq"

#1000 Read subsets for each file from the middle of the file
#zcat /projects/bgmp/shared/2017_sequencing/1294_S1_L008_R4_001.fastq.gz | head -n4000000 | tail -n4000 > R4_1000r.fq
file1000 = "testFiles/R1_1000r.fq"
file2000 = "testFiles/R2_1000r.fq"
file3000 = "testFiles/R3_1000r.fq"
file4000 = "testFiles/R4_1000r.fq"

In [4]:
fastqLines = 4
end = False
N = 0
IH = 0
QS = 0
reads = 0

fh1 = open(file1000, "r")
fh2 = open(file2000, "r")
fh3 = open(file3000, "r")
fh4 = open(file4000, "r")

read1 = [None] * 4
indexR1 = [None] * 4
indexR2 = [None] * 4
read2 = [None] * 4

indices = {
    'GTAGCGTA': 0, 'CGATCGAT': 0, 'GATCAAGG': 0, 'AACAGCGA': 0,
    'TAGCCATG': 0, 'CGGTAATC': 0, 'CTCTGGAT': 0, 'TACCGGAT': 0, 
    'CTAGCTCA': 0, 'CACTTCAC': 0, 'GCTACTCT': 0, 'ACGATCAG': 0, 
    'TATGGCAC': 0, 'TGTTCCGT': 0, 'GTCCTAAG': 0, 'TCGACAAG': 0,
    'TCTTCGAC': 0, 'ATCATGCG': 0, 'ATCGTGGT': 0, 'TCGAGAGT': 0,
    'TCGGATTC': 0, 'GATCTTGC': 0, 'AGAGTCCA': 0, 'AGGATAGC': 0,
    'SNP': 0
}

while end != True:
    flag = 0
    for x in range(fastqLines):
        read1[x] = readInLine(fh1)
        if read1[x] == "":
            end = True
            break
        indexR1[x] = readInLine(fh2)
        indexR2[x] = readInLine(fh3)
        read2[x] = readInLine(fh4)
        
        if x == 2:
            read1[2] = read1[2] + " " + indexR1[1]
            index1 = indexR1[1]
            read2[2] = read2[2] + " " + indexR2[1]
            index2 = indexR2[1]
    if end == True:
        break

    reads = reads + 1 
        
    if (containsN(index1)) or (containsN(index2)):
        flag = flag + 1
        N = N + 1
    
    if not(matching(index1, index2)):
        flag = flag + 2
        IH = IH + 1
    
    if not(compareMeans(read1[1])) or not(compareMeans(read1[1])):
        flag = flag + 4
        QS = QS + 1
    
    if flag == 0:
        if index1 in indices.keys():
            indices[index1] = indices[index1] + 1
            writeToGoodFile('outputFiles/%s_R1.fastq' % index1, read1)
            writeToGoodFile('outputFiles/%s_R2.fastq' % index1, read2)
        else:
            flag = flag + 8
            indices["SNP"] = indices["SNP"] + 1
            writeToBadFile('outputFiles/rejects_R1.fastq', read1, flag)
            writeToBadFile('outputFiles/rejects_R2.fastq', read1, flag)
    else:
        writeToBadFile('outputFiles/rejects_R1.fastq', read1, flag)
        writeToBadFile('outputFiles/rejects_R2.fastq', read2, flag)

for key in indices:
    indices[key] = round(((indices[key] / reads) * 100), 2)
        
print("Reads: ", reads)
print("N's  : ", N)
print("IH   : ", IH)
print("QS   : ", QS)
print("\n")
print("% of reads for each index")
print(indices)

fh1.close()
fh2.close()
fh3.close()
fh4.close()

Reads:  1000
N's  :  4
IH   :  85
QS   :  0


% of reads for each index
{'GTAGCGTA': 1.8, 'CGATCGAT': 1.6, 'GATCAAGG': 2.6, 'AACAGCGA': 2.6, 'TAGCCATG': 2.3, 'CGGTAATC': 1.3, 'CTCTGGAT': 8.6, 'TACCGGAT': 21.7, 'CTAGCTCA': 5.1, 'CACTTCAC': 1.3, 'GCTACTCT': 1.7, 'ACGATCAG': 2.1, 'TATGGCAC': 3.6, 'TGTTCCGT': 4.6, 'GTCCTAAG': 2.6, 'TCGACAAG': 1.2, 'TCTTCGAC': 13.2, 'ATCATGCG': 3.0, 'ATCGTGGT': 1.5, 'TCGAGAGT': 2.9, 'TCGGATTC': 0.8, 'GATCTTGC': 0.7, 'AGAGTCCA': 2.7, 'AGGATAGC': 1.8, 'SNP': 0.1}


In [5]:
with open("outputFiles/summary.txt", "w") as out:
    out.write("Reads: ")
    out.write(str(reads))
    out.write("\n")
    out.write("N's  : ")
    out.write(str(N))
    out.write("\n")
    out.write("IH   : ")
    out.write(str(IH))
    out.write("\n")
    out.write("QS   : ")
    out.write(str(QS))
    out.write("\n")
    out.write("\n")
    out.write("% of reads for each index")
    out.write("\n")
    out.write(str(indices))

In [ ]:
indices = {
    'GTAGCGTA': 0, 'CGATCGAT': 0, 'GATCAAGG': 0, 'AACAGCGA': 0,
    'TAGCCATG': 0, 'CGGTAATC': 0, 'CTCTGGAT': 0, 'TACCGGAT': 0, 
    'CTAGCTCA': 0, 'CACTTCAC': 0, 'GCTACTCT': 0, 'ACGATCAG': 0, 
    'TATGGCAC': 0, 'TGTTCCGT': 0, 'GTCCTAAG': 0, 'TCGACAAG': 0,
    'TCTTCGAC': 0, 'ATCATGCG': 0, 'ATCGTGGT': 0, 'TCGAGAGT': 0,
    'TCGGATTC': 0, 'GATCTTGC': 0, 'AGAGTCCA': 0, 'AGGATAGC': 0,
    'SNP': 0
}
def openFile(file)
    file

for key in indices:
    openFile('%s_R1' % key)